In [1]:
import pandas as pd

train_data = pd.read_csv("train.csv")

In [2]:
print(train_data.head())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [3]:
# get training and test data
from sklearn.model_selection import train_test_split

y = train_data["SalePrice"]
X = train_data.drop("SalePrice",axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [4]:
# creating a pipeline to clean the data 
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb


numerical_transfomer = SimpleImputer(strategy = "mean")
categorical_transformer = Pipeline(
    steps = [
        ("imputer",SimpleImputer(strategy= "most_frequent")),
        ("encoder",OneHotEncoder(handle_unknown="ignore", sparse=False))
    ]
)

preprocessing = Pipeline(
    steps = [
        ("cat",categorical_transformer),
        ("num",numerical_transfomer)
        
    ]
)

model = xgb.XGBRegressor(n_estimators=100,learning_rate=0.1)

pipeline = Pipeline(
    steps = [
        ("preprocessing",preprocessing),
        ("model",model)
        
    ]
)

In [5]:
print(X_train.shape)
print(X.shape)
print(X_test.shape)

(1095, 80)
(1460, 80)
(365, 80)


In [6]:
# fitting and scoring model 
from sklearn.metrics import mean_absolute_error

pipeline.fit(X_train,y_train)
predictions = pipeline.predict(X_test)
score = mean_absolute_error(y_test,predictions)
print(score)

20653.28797089041


In [15]:
# run model on test data 
predictions = pipeline.predict(test_data)
Id = test_data["Id"]
print(type(Id))
print(type(predictions))

submission = pd.DataFrame(data=predictions,index=Id,columns=["SalePrice"])
submission.head()
submission.to_csv("submission.csv")


<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
